In [1]:
import bifacial_radiance
import os
from pathlib import Path
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
bifacial_radiance.__version__

'0.3.4+121.g94c7e72.dirty'

#### Control variables

In [3]:
smallsim = True
shamsul = False


In [4]:
testfolder = str(Path().resolve().parent.parent / 'bifacial_radiance' / 'TEMP' / 'FrameTest')

if not os.path.exists(testfolder):
    os.makedirs(testfolder)
    
print ("Your simulation will be stored in %s" % testfolder)


Your simulation will be stored in C:\Users\sayala\Documents\GitHub\bifacial_radiance\bifacial_radiance\TEMP\FrameTest


In [20]:
if shamsul:
    TMYtoread = r'C:\Users\sarefeen\Documents\RadianceScenes\Temp\SRRL_WeatherFile_TMY3_60_2020.csv'
else:
    TMYtoread = r'C:\Users\sayala\Documents\GitHub\RTCanalysis\TEMP\SRRL_WeatherFile_TMY3_60_2020_FIXED.csv'

In [6]:
#### Reading the weatherfile
'''
weatherfile = pd.read_csv(TMYtoread, header = 1)
weatherfile.head()
wf2 = weatherfile[weatherfile['DNI (W/m^2)'] == weatherfile['DNI (W/m^2)'].max()]
wf3 = weatherfile[weatherfile['Date (MM/DD/YYYY)']== '4/29/2020']
y = wf3['DNI (W/m^2)']
x = wf3['Time (HH:MM)']
plt.plot(x,y)
plt.xticks(rotation = 45)
weatherfile.groupby('Date (MM/DD/YYYY)')['DNI (W/m^2)'].sum().max()
''';

### Make Modules

In [7]:
simulationname = 'OmegaTestField'
moduletype_framed='Framed_Panel'
moduletype_simple='NotFramed_Panel'

numpanels = 1  
x = 1  
y = 2
lat=39.742 # NREL SSRL location
lon=-105.179 # NREL SSRL location
elev=1829
timezone=-7
axis_tilt=0
axis_azimuth=180
limit_angle=60
backtrack=True 
gcr=0.35
angledelta=0.01
numpanels=1
torquetube=True
diameter = 0.130175        # 5 1/8 in
torqueTubeMaterial='Metal_Grey'
tubetype='Round'

# for torquetube, the simulation 
axisofrotationTorqueTube = False  # This is particular to the NREL site.
hub_height = 1.35    
xgap = 0.01    # 1 cm
zgap = 0.05    # 1 inch of arm, + 1 3/16 of panel width on average ~ 0.055 m
pitch=5.7      # distance between rows

sensorsy = 3 # Increase sampling for edge 

In [8]:
# TorqueTube Parameters
axisofrotationTorqueTube=False
torqueTube = True
cellLevelModule = False

In [9]:
albedo = 0.2  #'grass'     # ground albedo


#this change is for smalling the simulation
if smallsim:
    nMods = 1 
    nRows = 1 
    sensorsy = 3
else:
    nMods = 20 
    nRows = 10 
    sensorsy = 9
cumulativesky = False

In [10]:
demo = bifacial_radiance.RadianceObj(simulationname, path = testfolder)  # Create a RadianceObj 'object'

path = C:\Users\sayala\Documents\GitHub\bifacial_radiance\bifacial_radiance\TEMP\FrameTest


In [11]:
frameParams = {'frame_material' : 'Metal_Grey', 
               'frame_thickness' : 0.003,
               'frame_z' : 0.03,
               'nSides_frame' : 4,
               'frame_width' : 0.05}


omegaParams = {'omega_material': 'litesoil',
                'x_omega1' : 0.08,
                'mod_overlap' : 0.03,
                'y_omega' : 0.75,
                'x_omega3' : 0.02,
                'omega_thickness' : 0.01,
                'inverted' : False}
moduleDict=demo.makeModule(name=moduletype_simple,x=x,y=y,numpanels = numpanels, xgap=xgap, zgap=zgap,
                            torquetube=torquetube, diameter=diameter, tubetype=tubetype, material=torqueTubeMaterial,
                            axisofrotationTorqueTube=axisofrotationTorqueTube, omegaParams = None, frameParams = None)

moduleDict=demo.makeModule(name=moduletype_framed,x=x,y=y,numpanels = numpanels, xgap=xgap, zgap=zgap,
                            torquetube=torquetube, diameter=diameter, tubetype=tubetype, material=torqueTubeMaterial,
                            axisofrotationTorqueTube=axisofrotationTorqueTube, omegaParams = omegaParams, frameParams = frameParams)


Module Name: NotFramed_Panel
Postion 1 Offsetfromaxis: 0
Module NotFramed_Panel updated in module.json

Module Name: Framed_Panel
it is getting in to make omega
Postion 1 Offsetfromaxis: 0
Module Framed_Panel updated in module.json


start: 2020-01-01 01:00:00
end: 2020-12-31 23:00:00
Saving file EPWs\tmy3_temp.csv, # points: 8760
Calculating Sun position for Metdata that is right-labeled  with a delta of -30 mins. i.e. 12 is 11:30 sunpos
 Assigned Albedo from metdata.albedo
Loading albedo, 8759 value(s), 0.247 avg
4672 nonzero albedo values.


## A. SIMPLE Module run

In [25]:
# Restricting run to one hour for speed, 'MM_DD_HH'
metdata = demo.readWeatherFile(TMYtoread)
demo.setGround() 
sceneDict = {'pitch': pitch,'hub_height':hub_height, 'nMods':nMods, 'nRows': nRows, 'sceney':y}  
trackerdict = demo.set1axis(metdata = metdata, limit_angle = limit_angle, backtrack = backtrack, 
                            gcr = gcr, cumulativesky = cumulativesky)
startdate = '20_01_01_12'      
enddate = '20_01_01_12'
trackerdict = demo.gendaylit1axis(startdate = startdate, enddate = enddate) 
trackerdict = demo.makeScene1axis(moduletype = moduletype_simple, sceneDict = sceneDict) 
trackerdict = demo.makeOct1axis()
result = demo.analysis1axis(customname='test')

print("\n TRACKER TILT:", demo.trackerdict['20_01_01_12_00']['surf_tilt']) # Sanity checkt of surface tilt



start: 2020-01-01 01:00:00
end: 2020-12-31 23:00:00
Saving file EPWs\tmy3_temp.csv, # points: 8760
Calculating Sun position for Metdata that is right-labeled  with a delta of -30 mins. i.e. 12 is 11:30 sunpos
 Assigned Albedo from metdata.albedo
Loading albedo, 8759 value(s), 0.247 avg
4672 nonzero albedo values.
Creating ~0 skyfiles.  Takes 1-2 minutes
Created 1 skyfiles in /skies/

Making ~1 .rad files for gendaylit 1-axis workflow (this takes a minute..)
1 Radfiles created in /objects/

Making 1 octfiles in root directory.
Created 1axis_20_01_01_12_00.oct


## B. With Frame Test

In [32]:
# Restricting run to one hour for speed, 'MM_DD_HH'
metdata = demo.readWeatherFile(TMYtoread)
demo.setGround() 
sceneDict = {'pitch': pitch,'hub_height':hub_height, 'nMods':nMods, 'nRows': nRows, 'sceney':y}  
trackerdict = demo.set1axis(metdata = metdata, limit_angle = limit_angle, backtrack = backtrack, 
                            gcr = gcr, cumulativesky = cumulativesky)
startdate = '20_01_01_12'      
enddate = '20_01_01_12'
trackerdict = demo.gendaylit1axis(startdate = startdate, enddate = enddate) 
trackerdict = demo.makeScene1axis(moduletype = moduletype_framed, sceneDict = sceneDict) 
trackerdict = demo.makeOct1axis()
print(demo.trackerdict['20_01_01_12_00']['surf_tilt'])
result = demo.analysis1axis(customname='test_B', sensorsy=sensorsy)


start: 2020-01-01 01:00:00
end: 2020-12-31 23:00:00
Saving file EPWs\tmy3_temp.csv, # points: 8760
Calculating Sun position for Metdata that is right-labeled  with a delta of -30 mins. i.e. 12 is 11:30 sunpos
 Assigned Albedo from metdata.albedo
Loading albedo, 8759 value(s), 0.247 avg
4672 nonzero albedo values.
Creating ~0 skyfiles.  Takes 1-2 minutes
Created 1 skyfiles in /skies/

Making ~1 .rad files for gendaylit 1-axis workflow (this takes a minute..)
1 Radfiles created in /objects/


## Compare Results

In [37]:
bifacial_radiance.load.read1Result('results\irr_1axis_20_01_01_12_00test.csv')

,x,y,z,rearZ,mattype,rearMat,Wm2Front,Wm2Back,Back/FrontRatio
0,0.771556,4.724416e-17,1.137531,1.116481,a0.0.a0.NotFramed_Panel.6457,a0.0.a0.NotFramed_Panel.2310,115.5523,20.386953,0.176429
1,0.580193,3.552657e-17,1.195672,1.174622,a0.0.a0.NotFramed_Panel.6457,a0.0.a0.NotFramed_Panel.2310,115.5622,20.237973,0.175125
2,0.388830,2.380898e-17,1.253812,1.232762,a0.0.a0.NotFramed_Panel.6457,a0.0.a0.NotFramed_Panel.2310,115.5721,20.088790,0.173819
3,0.197467,1.209139e-17,1.311953,1.290903,a0.0.a0.NotFramed_Panel.6457,a0.0.a0.NotFramed_Panel.2310,115.5820,19.939610,0.172513
4,0.006105,3.738079e-19,1.370093,1.349043,a0.0.a0.NotFramed_Panel.6457,a0.0.a0.NotFramed_Panel.2310,115.5920,12.307183,0.106470
5,-0.185258,-1.134378e-17,1.428234,1.407184,a0.0.a0.NotFramed_Panel.6457,a0.0.a0.NotFramed_Panel.2310,115.6019,19.483470,0.168538
6,-0.376621,-2.306137e-17,1.486374,1.465324,a0.0.a0.NotFramed_Panel.6457,a0.0.a0.NotFramed_Panel.2310,115.6118,20.338293,0.175917
7,-0.567983,-3.477895e-17,1.544514,1.523465,a0.0.a0.NotFramed_Panel.6457,a0.0.a0.NotFramed_Panel.2310,115.4579,20.520810,0.177733
8,-0.759346,-4.649654e-17,1.602655,1.581605,a0.0.a0.NotFramed_Panel.6457,a0.0.a0.NotFramed_Panel.2310,115.4655,20.779643,0.179963


In [40]:
bifacial_radiance.load.read1Result('results\irr_1axis_20_01_01_12_00test_B.csv')

,x,y,z,rearZ,mattype,rearMat,Wm2Front,Wm2Back,Back/FrontRatio
0,0.771556,4.724416e-17,1.137531,1.116481,a0.0.a0.Framed_Panel.6457,a0.0.a0.Framed_Panel.2310,115.6612,18.989073,0.164177
1,0.580193,3.552657e-17,1.195672,1.174622,a0.0.a0.Framed_Panel.6457,a0.0.a0.Framed_Panel.2310,115.6705,18.848463,0.162948
2,0.388830,2.380898e-17,1.253812,1.232762,a0.0.a0.Framed_Panel.6457,a0.0.a0.Framed_Panel.2310,115.6797,18.012183,0.155706
3,0.197467,1.209139e-17,1.311953,1.290903,a0.0.a0.Framed_Panel.6457,a0.0.a0.Framed_Panel.2310,115.6889,17.040757,0.147297
4,0.006105,3.738079e-19,1.370093,1.349043,a0.0.a0.Framed_Panel.6457,a0.0.a0.Framed_Panel.2310,115.6981,13.475497,0.116470
5,-0.185258,-1.134378e-17,1.428234,1.407184,a0.0.a0.Framed_Panel.6457,a0.0.a0.Framed_Panel.2310,115.7073,18.562733,0.160427
6,-0.376621,-2.306137e-17,1.486374,1.465324,a0.0.a0.Framed_Panel.6457,a0.0.a0.Framed_Panel.2310,115.7165,19.740280,0.170590
7,-0.567983,-3.477895e-17,1.544514,1.523465,a0.0.a0.Framed_Panel.6457,a0.0.a0.Framed_Panel.2310,115.5555,20.154020,0.174408
8,-0.759346,-4.649654e-17,1.602655,1.581605,a0.0.a0.Framed_Panel.6457,a0.0.a0.Framed_Panel.2310,115.5588,20.513297,0.177512


# C. Modify Scanning position, Not Framed

In [41]:
# Restricting run to one hour for speed, 'MM_DD_HH'
metdata = demo.readWeatherFile(TMYtoread)
demo.setGround() 
sceneDict = {'pitch': pitch,'hub_height':hub_height, 'nMods':nMods, 'nRows': nRows, 'sceney':y}  
trackerdict = demo.set1axis(metdata = metdata, limit_angle = limit_angle, backtrack = backtrack, 
                            gcr = gcr, cumulativesky = cumulativesky)
startdate = '20_01_01_12'      
enddate = '20_01_01_12'
trackerdict = demo.gendaylit1axis(startdate = startdate, enddate = enddate) 
trackerdict = demo.makeScene1axis(moduletype = moduletype_simple, sceneDict = sceneDict) 
trackerdict = demo.makeOct1axis()
print(demo.trackerdict['20_01_01_12_00']['surf_tilt'])

scanpoints = int(0.05/0.005)     # sample 5 cm from the edge, with a resolution of 0.005 mm 
frame_thickness = 0.01
modscanBack = {}
for ii in range(0, 2):
    modscanBack['ystart']  = x/2.0 - (frame_thickness + 0.001) - 0.005*ii # (adding frame thicknes plus 1 mm so it does not overlay exactly) 
    result = demo.analysis1axis(modscanfront=modscanBack, modscanback=modscanBack, relative = False, customname='_test_C_pos_'+str(ii))

    

start: 2020-01-01 01:00:00
end: 2020-12-31 23:00:00
Saving file EPWs\tmy3_temp.csv, # points: 8760
Calculating Sun position for Metdata that is right-labeled  with a delta of -30 mins. i.e. 12 is 11:30 sunpos
 Assigned Albedo from metdata.albedo
Loading albedo, 8759 value(s), 0.247 avg
4672 nonzero albedo values.
Creating ~0 skyfiles.  Takes 1-2 minutes
Created 1 skyfiles in /skies/

Making ~1 .rad files for gendaylit 1-axis workflow (this takes a minute..)
1 Radfiles created in /objects/

Making 1 octfiles in root directory.
Created 1axis_20_01_01_12_00.oct
16.9
Linescan in process: 1axis_20_01_01_12_00_test_C_pos_0_Front
Linescan in process: 1axis_20_01_01_12_00_test_C_pos_0_Back
Saved: results\irr_1axis_20_01_01_12_00_test_C_pos_0.csv
Index: 20_01_01_12_00. Wm2Front: 115.50092222222221. Wm2Back: 19.989597777777778
Saving a cumulative-results file in the main simulation folder.This adds up by sensor location the irradiance over all hours or configurations considered.

Saving Cumulati

In [43]:
bifacial_radiance.load.read1Result('results\irr_1axis_20_01_01_12_00_test_C_pos_0.csv')

,x,y,z,rearZ,mattype,rearMat,Wm2Front,Wm2Back,Back/FrontRatio
0,0.771556,0.489,1.137531,1.116481,a0.0.a0.NotFramed_Panel.6457,a0.0.a0.NotFramed_Panel.2310,115.4540,19.633207,0.170051
1,0.580193,0.489,1.195672,1.174622,a0.0.a0.NotFramed_Panel.6457,a0.0.a0.NotFramed_Panel.2310,115.4722,19.610517,0.169827
2,0.388830,0.489,1.253812,1.232762,a0.0.a0.NotFramed_Panel.6457,a0.0.a0.NotFramed_Panel.2310,115.4903,19.100007,0.165380
3,0.197467,0.489,1.311953,1.290903,a0.0.a0.NotFramed_Panel.6457,a0.0.a0.NotFramed_Panel.2310,115.5085,18.879583,0.163446
4,0.006105,0.489,1.370093,1.349043,a0.0.a0.NotFramed_Panel.6457,a0.0.a0.NotFramed_Panel.2310,115.5267,21.570343,0.186711
5,-0.185258,0.489,1.428234,1.407184,a0.0.a0.NotFramed_Panel.6457,a0.0.a0.NotFramed_Panel.2310,115.5449,19.867337,0.171943
6,-0.376621,0.489,1.486374,1.465324,a0.0.a0.NotFramed_Panel.6457,a0.0.a0.NotFramed_Panel.2310,115.5630,20.193737,0.174741
7,-0.567983,0.489,1.544514,1.523465,a0.0.a0.NotFramed_Panel.6457,a0.0.a0.NotFramed_Panel.2310,115.4717,20.387540,0.176557
8,-0.759346,0.489,1.602655,1.581605,a0.0.a0.NotFramed_Panel.6457,a0.0.a0.NotFramed_Panel.2310,115.4770,20.664110,0.178944


In [44]:
bifacial_radiance.load.read1Result('results\irr_1axis_20_01_01_12_00_test_C_pos_1.csv')

,x,y,z,rearZ,mattype,rearMat,Wm2Front,Wm2Back,Back/FrontRatio
0,0.8,0.484,1.371,1.349,a0.0.a0.NotFramed_Panel.6457,sky,115.560700,41.041610,0.355149
1,0.6,0.484,1.371,1.349,a0.0.a0.NotFramed_Panel.6457,sky,115.574300,41.041610,0.355107
2,0.4,0.484,1.371,1.349,a0.0.a0.NotFramed_Panel.6457,sky,115.587900,41.041610,0.355065
3,0.2,0.484,1.371,1.349,a0.0.a0.NotFramed_Panel.6457,sky,115.601600,41.041610,0.355023
4,0.0,0.484,1.371,1.349,a0.0.a0.NotFramed_Panel.6457,a0.0.a0.NotFramed_Panel.2310,115.615200,22.827807,0.197445
5,-0.2,0.484,1.371,1.349,groundplane,a0.0.a0.NotFramed_Panel.2310,91.878843,20.663547,0.224897
6,-0.4,0.484,1.371,1.349,groundplane,a0.0.a0.NotFramed_Panel.2310,93.109193,20.763367,0.222998
7,-0.6,0.484,1.371,1.349,groundplane,a0.0.a0.NotFramed_Panel.2310,95.548213,21.057683,0.220386
8,-0.8,0.484,1.371,1.349,groundplane,a0.0.a0.NotFramed_Panel.2310,97.184197,21.026097,0.216351


## D. Modifying Scanning Position with Frames

In [46]:
# Restricting run to one hour for speed, 'MM_DD_HH'
metdata = demo.readWeatherFile(TMYtoread)
demo.setGround() 
sceneDict = {'pitch': pitch,'hub_height':hub_height, 'nMods':nMods, 'nRows': nRows, 'sceney':y}  
trackerdict = demo.set1axis(metdata = metdata, limit_angle = limit_angle, backtrack = backtrack, 
                            gcr = gcr, cumulativesky = cumulativesky)
startdate = '20_01_01_12'      
enddate = '20_01_01_12'
trackerdict = demo.gendaylit1axis(startdate = startdate, enddate = enddate) 
trackerdict = demo.makeScene1axis(moduletype = moduletype_framed, sceneDict = sceneDict) 
trackerdict = demo.makeOct1axis()
print(demo.trackerdict['20_01_01_12_00']['surf_tilt'])

scanpoints = int(0.05/0.005)     # sample 5 cm from the edge, with a resolution of 0.005 mm 
frame_thickness = 0.01
modscanBack = {}
for ii in range(0, 2):
    modscanBack['ystart']  = x/2.0 - (frame_thickness + 0.001) - 0.005*ii # (adding frame thicknes plus 1 mm so it does not overlay exactly) 
    result = demo.analysis1axis(modscanfront=modscanBack, modscanback=modscanBack, relative = False, customname='_test_D_pos_'+str(ii))

    

start: 2020-01-01 01:00:00
end: 2020-12-31 23:00:00
Saving file EPWs\tmy3_temp.csv, # points: 8760
Calculating Sun position for Metdata that is right-labeled  with a delta of -30 mins. i.e. 12 is 11:30 sunpos
 Assigned Albedo from metdata.albedo
Loading albedo, 8759 value(s), 0.247 avg
4672 nonzero albedo values.
Creating ~0 skyfiles.  Takes 1-2 minutes
Created 1 skyfiles in /skies/

Making ~1 .rad files for gendaylit 1-axis workflow (this takes a minute..)
1 Radfiles created in /objects/

Making 1 octfiles in root directory.
Created 1axis_20_01_01_12_00.oct
16.9
Linescan in process: 1axis_20_01_01_12_00_test_D_pos_0_Front
Linescan in process: 1axis_20_01_01_12_00_test_D_pos_0_Back
Saved: results\irr_1axis_20_01_01_12_00_test_D_pos_0.csv
Index: 20_01_01_12_00. Wm2Front: 115.47452222222222. Wm2Back: 2.5800401481481483
Saving a cumulative-results file in the main simulation folder.This adds up by sensor location the irradiance over all hours or configurations considered.

Saving Cumulati

In [47]:
bifacial_radiance.load.read1Result('results\irr_1axis_20_01_01_12_00_test_D_pos_0.csv')

,x,y,z,rearZ,mattype,rearMat,Wm2Front,Wm2Back,Back/FrontRatio
0,0.771556,0.489,1.137531,1.116481,a0.0.a0.Framed_Panel.6457,a0.0.a0.Framed_Panel.2310,115.4256,2.456016,0.021278
1,0.580193,0.489,1.195672,1.174622,a0.0.a0.Framed_Panel.6457,a0.0.a0.Framed_Panel.2310,115.4362,2.485982,0.021535
2,0.388830,0.489,1.253812,1.232762,a0.0.a0.Framed_Panel.6457,a0.0.a0.Framed_Panel.2310,115.4467,2.429947,0.021048
3,0.197467,0.489,1.311953,1.290903,a0.0.a0.Framed_Panel.6457,a0.0.a0.Framed_Panel.2310,115.4573,2.574326,0.022297
4,0.006105,0.489,1.370093,1.349043,a0.0.a0.Framed_Panel.6457,a0.0.a0.Framed_Panel.2310,115.4678,2.429487,0.021040
5,-0.185258,0.489,1.428234,1.407184,a0.0.a0.Framed_Panel.6457,a0.0.a0.Framed_Panel.2310,115.4784,2.738479,0.023714
6,-0.376621,0.489,1.486374,1.465324,a0.0.a0.Framed_Panel.6457,a0.0.a0.Framed_Panel.2310,115.4889,2.714885,0.023508
7,-0.567983,0.489,1.544514,1.523465,a0.0.a0.Framed_Panel.6457,a0.0.a0.Framed_Panel.2310,115.5318,2.790090,0.024150
8,-0.759346,0.489,1.602655,1.581605,a0.0.a0.Framed_Panel.6457,a0.0.a0.Framed_Panel.2310,115.5380,2.601149,0.022513


In [48]:
bifacial_radiance.load.read1Result('results\irr_1axis_20_01_01_12_00_test_C_pos_1.csv')

,x,y,z,rearZ,mattype,rearMat,Wm2Front,Wm2Back,Back/FrontRatio
0,0.8,0.484,1.371,1.349,a0.0.a0.Framed_Panel.6457,sky,115.469500,41.041610,0.355429
1,0.6,0.484,1.371,1.349,a0.0.a0.Framed_Panel.6457,sky,115.483600,41.041610,0.355386
2,0.4,0.484,1.371,1.349,a0.0.a0.Framed_Panel.6457,sky,115.497600,41.041610,0.355343
3,0.2,0.484,1.371,1.349,a0.0.a0.Framed_Panel.6457,sky,115.511600,41.041610,0.355300
4,0.0,0.484,1.371,1.349,a0.0.a0.Framed_Panel.6457,a0.0.a0.Framed_Panel.2310,115.525500,2.847268,0.024646
5,-0.2,0.484,1.371,1.349,a0.0.mod_adj.2310,a0.0.mod_adj.2310,114.974200,17.578170,0.152887
6,-0.4,0.484,1.371,1.349,groundplane,a0.0.a0.frameleg.2310,90.988897,19.514040,0.214464
7,-0.6,0.484,1.371,1.349,groundplane,a0.0.a0.frameleg.2310,92.683813,20.262050,0.218612
8,-0.8,0.484,1.371,1.349,groundplane,a0.0.a0.frameleg.2310,96.431143,20.583270,0.213448
